In [1]:
import tensorcircuit as tc
import tensorflow as ts 
import numpy as np
import math
from numpy import linalg as LA
K = tc.set_backend("tensorflow")

dt=0.01
n=1/dt
t=0
h=0.5

nq=2
c1=tc.Circuit(nq)
c2=tc.Circuit(nq)
H = 0
theta = math.pi/2
min_w = 999
w=0
v=0
I = tc.gates._i_matrix
X = tc.gates._x_matrix  
Y = tc.gates._y_matrix  
Z = tc.gates._z_matrix 

def i_iterate_multiplication(u):
    tmp = 1
    for i in range(u):
        tmp = np.kron(I, tmp)
    return tmp

def zzi(i):
     return np.kron(np.kron(i_iterate_multiplication(i), np.kron(Z, Z)), i_iterate_multiplication(nq-2-i))
    
def xi(i):
    return h * (np.kron(np.kron(i_iterate_multiplication(i), X), i_iterate_multiplication(nq-1-i)))

2022-07-27 14:06:55.705307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 14:06:55.705323: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-27 14:06:58.078913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-27 14:06:58.078931: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-27 14:06:58.078941: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (imasonliu-LC2): /proc/driver/nvidia/version does not exist
2022-07-27 14:06:58.079655: I tensorflow/core/platform/cpu_feat

In [2]:
for i in range(0, nq-1):
    H = H - zzi(i)
for i in range(0, nq):
    H = H + xi(i)
print(H)

[[-1.   0.5  0.5  0. ]
 [ 0.5  1.   0.   0.5]
 [ 0.5  0.   1.   0.5]
 [ 0.   0.5  0.5 -1. ]]


In [3]:
#w, v = LA.eig(H)
#for i in range(0, len(w)):
#        if w[i] < min_w:
#            min_w = w[i]
            
h1=tc.gates.exponential_gate(H,1)
c1.any(0,1,unitary=h1)
print(c1)
n=int(n)
for i in range(n):
    t+=dt
    zz=tc.gates.exponential_gate(zzi(j),dt)
    for j in range(nq-1):
        
        c2.any(0,1,unitary=zz)
    for j in range(nq):
        c2.any(0,1,unitary=xi(j)*dt)     
print(c1.state())
print(c2.state())

tf.Tensor(
[ 0.34812307+0.7699635j   0.        -0.34922802j  0.        -0.349228j
 -0.1921793 -0.0715075j ], shape=(4,), dtype=complex64)


InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a complex64 tensor [Op:MatMul]